In [1]:
import torch
import h5py
import yaml
import os

from laplacian import *

In [2]:
import sys
import importlib.util

solver_dir = "/home/ubuntu/SML/solver"

sys.path.append(solver_dir)

file_path = "/home/ubuntu/SML/solver/solver.py"
spec = importlib.util.spec_from_file_location("solver", file_path)
solver = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver)

In [3]:
E = 113.8e9
nu = 0.342
rho = 4.47e-3

In [4]:
device = "cuda:2"
dtype = torch.float32

In [5]:
files = os.listdir("/data/SimJEB/small_ver_25/preprocessed/")
print(files)

['sample_19.h5', 'sample_35.h5', 'sample_40.h5', 'sample_0.h5', 'sample_27.h5', 'sample_14.h5', 'sample_38.h5', 'sample_10.h5', 'sample_12.h5', 'sample_29.h5', 'sample_28.h5', 'sample_23.h5', 'sample_8.h5', 'sample_63.h5', 'sample_4.h5', 'sample_16.h5', 'sample_6.h5', 'sample_21.h5', 'sample_20.h5', 'sample_15.h5', 'sample_30.h5', 'sample_33.h5', 'sample_62.h5', 'sample_22.h5', 'sample_9.h5']


In [6]:
i = 18

idx = files[i].split('_')[1].split('.')[0]
print(idx)

with open(f'/data/SimJEB/boundary/{idx}.yaml') as file:
    boundary = yaml.safe_load(file)

with h5py.File(f"/data/SimJEB/small_ver_25/preprocessed/"+files[i], 'r') as f:
    coords = torch.tensor(f['coords'][:])
    elements = torch.tensor(f['elements'][:])
    displacement = torch.tensor(f['outputs'][:])[:,0,:3]
    stress = torch.tensor(f['outputs'][:])[:,0,-1] * 1e6   # scale unit (MPa to Pa)

coords = coords.to(device=device, dtype=dtype)
elements = elements.to(device=device, dtype=torch.int32)
displacement = displacement.to(device=device, dtype=dtype)
stress = stress.to(device=device, dtype=dtype)

20


In [7]:
print(coords.shape, elements.shape, displacement.shape, stress.shape)

torch.Size([57168, 3]) torch.Size([267823, 4]) torch.Size([57168, 3]) torch.Size([57168])


In [8]:
edge, tetra_edge_id, tetra_edge_orientation = tetrahedral_to_full_edge_data(elements, device)
print(edge.shape, tetra_edge_id.shape, tetra_edge_orientation.shape)

torch.Size([352667, 2]) torch.Size([267823, 6]) torch.Size([267823, 6])


In [9]:
coord_gradient = compute_target_gradient(coords, edge)
print(coord_gradient.shape)

torch.Size([352667, 3])


In [10]:
loss = compute_gradient_field_loop_conservation_loss(coord_gradient, tetra_edge_id, tetra_edge_orientation, device, dtype)
print(loss.shape)

torch.Size([1071292, 3])


In [11]:
element_vm_stress_calculated = solver.compute_element_stress(coords, elements, displacement, E, nu, "c3d4", device, dtype)[1]

In [12]:
node_vm_stress_calculated = solver.compute_node_vm_stress(coords, elements, element_vm_stress_calculated, device, dtype)

In [13]:
# solver.visualize_node_with_value(coords, stress)

In [14]:
displacement_gradient = compute_target_gradient(displacement, edge)

In [15]:
element_vm_stress_calculated_laplacian = compute_element_stress_by_displacement_gradient(coords, elements, displacement_gradient, tetra_edge_id, tetra_edge_orientation, E, nu, device, dtype)[1]
print(element_vm_stress_calculated_laplacian.shape)

torch.Size([267823])


In [16]:
node_vm_stress_calculated_laplacian = solver.compute_node_vm_stress(coords, elements, element_vm_stress_calculated_laplacian, device, dtype)

In [17]:
# solver.visualize_node_with_value(coords, node_vm_stress_calculated_laplacian)

In [18]:
free_mem, total_mem = torch.cuda.mem_get_info(device)
print(f"Free memory on {device}: {free_mem / (1024**3):.2f} GB")
print(f"Total memory on {device}: {total_mem / (1024**3):.2f} GB")

Free memory on cuda:2: 77.98 GB
Total memory on cuda:2: 79.15 GB


In [19]:
coords_recovered = recover_nodal_field_spanning_tree(edge, coord_gradient, coords.shape[0], torch.tensor([0]), device)
print(coords_recovered.shape)

torch.Size([57168, 3])


In [20]:
displacement_recovered = recover_nodal_field_spanning_tree(edge, displacement_gradient, coords.shape[0], torch.tensor([0]), device)
element_stress_recovered = solver.compute_element_stress(coords, elements, displacement_recovered, E, nu, "c3d4", device, dtype)[1]
node_vm_stress_recovered = solver.compute_node_vm_stress(coords, elements, element_stress_recovered, device, dtype)

In [21]:
# solver.visualize_node_with_value(coords_recovered, node_vm_stress_recovered, title="Recovered Coords, with vm stress calculated from recovered displacement")

In [22]:
edge_edge = compute_edge_edge(tetra_edge_id, device)
print(edge_edge.shape)

torch.Size([2034603, 2])


In [23]:
edge_centroid = coords[edge].mean(dim=1)

rbe2_list = [rbe2['slaves'] for rbe2 in boundary['rbe2']]
rbe3_list = [rbe3['slaves'] for rbe3 in boundary['rbe3']]
edge_rbe2_mask_list = []
edge_rbe3_mask_list = []
for rbe2 in rbe2_list:
    rbe2_ids = torch.tensor(rbe2, device=device)
    mask = torch.isin(edge[:, 0], rbe2_ids) & torch.isin(edge[:, 1], rbe2_ids)
    edge_rbe2_mask = torch.zeros(edge.shape[0], dtype=torch.bool, device=device)
    edge_rbe2_mask |= mask
    edge_rbe2_mask_list.append(mask)
for rbe3 in rbe3_list:
    rbe3_ids = torch.tensor(rbe3, device=device)
    mask = torch.isin(edge[:, 0], rbe3_ids) & torch.isin(edge[:, 1], rbe3_ids)
    edge_rbe3_mask = torch.zeros(edge.shape[0], dtype=torch.bool, device=device)
    edge_rbe3_mask |= mask
    edge_rbe3_mask_list.append(mask)

In [24]:
# solver.visualize_target_nodes(edge_centroid[edge_rbe2_mask_list[0]], marker_size=5, target_marker_size=5)

In [25]:
hop = compute_hop_iteration_for_edge(edge_edge, edge.shape[0], edge_rbe2_mask_list[0], device)
print(hop.shape)

torch.Size([352667])


In [26]:
edge_length = torch.norm(coord_gradient, dim=-1)
hop_length = compute_length_sum_for_edge(edge_edge, edge.shape[0], edge_rbe2_mask_list[0], edge_length, device)
print(hop_length.shape)

torch.Size([352667])


In [27]:
rbe2_hop_length = []
for i in range(len(edge_rbe2_mask_list)):
    hop_length = compute_length_sum_for_edge(edge_edge, edge.shape[0], edge_rbe2_mask_list[i], edge_length, device)
    rbe2_hop_length.append(hop_length)
rbe3_hop_length = []
for i in range(len(edge_rbe3_mask_list)):
    hop_length = compute_length_sum_for_edge(edge_edge, edge.shape[0], edge_rbe3_mask_list[i], edge_length, device)
    rbe3_hop_length.append(hop_length)
rbe2_hop_length = torch.exp(torch.log(torch.stack(rbe2_hop_length, dim=0)).mean(dim=0))
rbe3_hop_length = torch.exp(torch.log(torch.stack(rbe3_hop_length, dim=0)).mean(dim=0))
print(rbe2_hop_length.shape)
print(rbe3_hop_length.shape)

torch.Size([352667])
torch.Size([352667])


In [28]:
x = torch.cat([coord_gradient, edge_length.unsqueeze(-1)], dim=-1)
print(x.shape)

torch.Size([352667, 4])


In [35]:
# solver.visualize_node_with_value(edge_centroid, hop_length, title="Edge domain hop lnength from RBE2 0")